In [ ]:
from tkinter import *
from tkinter import ttk
from tkinter import messagebox
import random
import mysql.connector # Make sure you have installed: pip install mysql-connector-python
from ttkthemes import ThemedTk # Make sure you have installed: pip install ttkthemes

class Hospital:
    def __init__(self, root):
        self.root = root
        self.root.title("Hospital Management System")
        self.root.geometry("1540x850+0+0")
        
        # --- Database Connection Configuration ---
        self.DB_PASSWORD = 'your mysql password' # <--- IMPORTANT: CHANGE THIS TO YOUR ACTUAL PASSWORD!

        # --- Data Variables ---
        self.RefNo_var = StringVar()
        self.NameTablet_var = StringVar(value="Nice")
        self.Dose_var = StringVar()
        self.NumTablet_var = StringVar()
        self.Lot_var = StringVar()
        self.IssueDate_var = StringVar()
        self.ExpDate_var = StringVar()
        self.DailyDose_var = StringVar()
        self.SideEffect_var = StringVar()
        self.FurtherInfo_var = StringVar()
        self.BloodPressure_var = StringVar()
        self.Storage_var = StringVar()
        self.Medication_var = StringVar()
        self.PatientID_var = StringVar()
        self.NHSNo_var = StringVar()
        self.PatientName_var = StringVar()
        self.DOB_var = StringVar()
        self.Address_var = StringVar()
        
        self.generate_reference_no()

        # =======================================TITLE================================================
        lbltitle = Label(self.root, bd=10, relief=RIDGE, text="HOSPITAL MANAGEMENT SYSTEM", fg="#A30000", bg="#EBEBEB", font=("times new roman", 40, "bold"))
        lbltitle.pack(side=TOP, fill=X)

        # =======================================MAIN FRAME================================================
        Dataframe = Frame(self.root, bd=15, relief=RIDGE, bg="#F0F0F0")
        Dataframe.place(x=0, y=85, width=1530, height=450)

        # LEFT FRAME (Patient Information)
        DataframeLeft = LabelFrame(Dataframe, bd=8, padx=20, relief=RIDGE, font=("times new roman", 14, "bold"), text="Patient Information", fg="#333333")
        DataframeLeft.place(x=8, y=5, width=980, height=420)

        # RIGHT FRAME (Prescription)
        DataframeRight = LabelFrame(Dataframe, bd=8, padx=15, relief=RIDGE, font=("times new roman", 14, "bold"), text="Prescription", fg="#333333")
        DataframeRight.place(x=995, y=5, width=490, height=420)

        # =========================BUTTON FRAME=================================
        Buttonframe = Frame(self.root, bd=10, relief=RIDGE, bg="#F0F0F0")
        Buttonframe.place(x=0, y=540, width=1530, height=70)

        # =========================DETAILS FRAME (FOR TREEVIEW)=================================
        Detailsframe = Frame(self.root, bd=10, relief=RIDGE, bg="white")
        Detailsframe.place(x=0, y=615, width=1530, height=230)

        # ===========================WIDGETS IN LEFT FRAME====================================
        
        # Left side fields
        labels = [
            ("Names Of Tablet:", self.NameTablet_var, ttk.Combobox, ("Nice", "Cororna vaccine", "Acetaminophen", "Adderal")),
            ("Reference No:", self.RefNo_var, Entry),
            ("Dose:", self.Dose_var, Entry),
            ("Number Of Tablet:", self.NumTablet_var, Entry),
            ("Lot:", self.Lot_var, Entry),
            ("Issue Date:", self.IssueDate_var, Entry),
            ("Exp Date:", self.ExpDate_var, Entry),
            ("Daily Dose:", self.DailyDose_var, Entry),
            ("Side Effect:", self.SideEffect_var, Entry),
        ]
        
        # Right side fields
        details = [
            ("Further Information:", self.FurtherInfo_var, Entry),
            ("Blood Pressure:", self.BloodPressure_var, Entry),
            ("Storage Advice:", self.Storage_var, Entry),
            ("Medication:", self.Medication_var, Entry),
            ("Patient Id:", self.PatientID_var, Entry),
            ("NHS Number:", self.NHSNo_var, Entry),
            ("Patient Name:", self.PatientName_var, Entry),
            ("Date Of Birth:", self.DOB_var, Entry),
            ("Patient Address:", self.Address_var, Entry),
        ]

        # Populate Left side (Column 0 and 1)
        for i, (text, var, widget_type, *values) in enumerate(labels):
            lbl = Label(DataframeLeft, text=text, font=("arial", 12, "bold"), padx=2, pady=8)
            lbl.grid(row=i, column=0, sticky=W)
            
            if widget_type == ttk.Combobox:
                widget = ttk.Combobox(DataframeLeft, font=("arial", 12, "bold"), width=33, textvariable=var, state='readonly')
                widget["values"] = values[0]
                widget.current(0)
            else:
                widget = Entry(DataframeLeft, font=("arial", 13, "bold"), width=35, textvariable=var)
            
            widget.grid(row=i, column=1, padx=5, pady=2)

        # Populate Right side (Column 2 and 3)
        for i, (text, var, widget_type) in enumerate(details):
            lbl = Label(DataframeLeft, text=text, font=("arial", 12, "bold"), padx=2, pady=8)
            lbl.grid(row=i, column=2, sticky=W, padx=(20, 0))
            
            widget = Entry(DataframeLeft, font=("arial", 13, "bold"), width=35, textvariable=var)
            widget.grid(row=i, column=3, padx=5, pady=2)

        # Prescription Text Area
        self.txtPrescription = Text(DataframeRight, font=("arial", 12), width=50, height=18, padx=5, pady=5, bd=4, relief=SUNKEN)
        self.txtPrescription.grid(row=0, column=0)

        # =========================BUTTON WIDGETS=================================
        
        btnSave = ttk.Button(Buttonframe, text="Save/Update", style='TButton', command=self.iSaveOrUpdate)
        btnSave.grid(row=0, column=0, padx=10, pady=5)
        
        btnView = ttk.Button(Buttonframe, text="View Records", style='TButton', command=self.iView)
        btnView.grid(row=0, column=1, padx=10, pady=5)
        
        btnClear = ttk.Button(Buttonframe, text="Clear Fields", style='TButton', command=self.iClear)
        btnClear.grid(row=0, column=2, padx=10, pady=5)
        
        btnDelete = ttk.Button(Buttonframe, text="Delete Record", style='TButton', command=self.iDelete)
        btnDelete.grid(row=0, column=3, padx=10, pady=5)

        btnExit = ttk.Button(Buttonframe, text="Exit", style='TButton', command=self.iExit)
        btnExit.grid(row=0, column=4, padx=10, pady=5)
        
        # =========================TREEVIEW (DETAILS FRAME)=================================
        
        scroll_x = ttk.Scrollbar(Detailsframe, orient=HORIZONTAL)
        scroll_y = ttk.Scrollbar(Detailsframe, orient=VERTICAL)
        
        self.record_tree = ttk.Treeview(Detailsframe, columns=("ref", "tablet", "patient_id", "name", "dob", "dose"), show='headings', xscrollcommand=scroll_x.set, yscrollcommand=scroll_y.set)
        
        scroll_x.pack(side=BOTTOM, fill=X)
        scroll_y.pack(side=RIGHT, fill=Y)
        scroll_x.config(command=self.record_tree.xview)
        scroll_y.config(command=self.record_tree.yview)

        self.record_tree.heading("ref", text="Reference No")
        self.record_tree.heading("tablet", text="Tablet Name")
        self.record_tree.heading("patient_id", text="Patient ID")
        self.record_tree.heading("name", text="Patient Name")
        self.record_tree.heading("dob", text="DOB")
        self.record_tree.heading("dose", text="Dose")
        
        self.record_tree.column("ref", width=150)
        self.record_tree.column("tablet", width=250)
        self.record_tree.column("patient_id", width=150)
        self.record_tree.column("name", width=250)
        self.record_tree.column("dob", width=150)
        self.record_tree.column("dose", width=100)
        
        self.record_tree.pack(fill='both', expand=True, padx=5, pady=5)
        
        self.record_tree.bind("<ButtonRelease-1>", self.iLoadDataFromTree)

    # =======================================FUNCTIONS================================================

    def db_connect(self):
        try:
            conn = mysql.connector.connect(
                host='localhost',
                user='root',
                password=self.DB_PASSWORD,
                database='hospital_db'
            )
            return conn
        except mysql.connector.Error as err:
            messagebox.showerror("Database Error", f"Failed to connect to MySQL. Ensure server is running and password is correct.\nError: {err}")
            return None

    def generate_reference_no(self):
        ref = str(random.randint(1000000, 9999999))
        self.RefNo_var.set("REF" + ref)

    def iSaveOrUpdate(self):
        ref_no = self.RefNo_var.get()
        if not ref_no:
            messagebox.showerror("Input Error", "Reference No cannot be empty.")
            return
            
        conn = self.db_connect()
        if conn is None: return

        try:
            cursor = conn.cursor()
            
            # 1. Gather ALL data from fields (excluding ref_no and prescription_text for initial tuple)
            data = (
                self.NameTablet_var.get(), self.Dose_var.get(), self.NumTablet_var.get(), self.Lot_var.get(), 
                self.IssueDate_var.get(), self.ExpDate_var.get(), self.DailyDose_var.get(), 
                self.SideEffect_var.get(), self.FurtherInfo_var.get(), self.BloodPressure_var.get(), 
                self.Storage_var.get(), self.Medication_var.get(), self.PatientID_var.get(), 
                self.NHSNo_var.get(), self.PatientName_var.get(), self.DOB_var.get(), 
                self.Address_var.get(), self.txtPrescription.get("1.0", END).strip()
            )
            
            # 2. CHECK if record exists using Reference No
            check_query = "SELECT reference_no FROM patient_records WHERE reference_no = %s"
            cursor.execute(check_query, (ref_no,))
            
            if cursor.fetchone():
                # ================= UPDATE LOGIC =================
                update_query = """
                UPDATE patient_records SET
                    name_of_tablet=%s, dose=%s, num_of_tablets=%s, lot_no=%s, issue_date=%s, exp_date=%s,
                    daily_dose=%s, side_effect=%s, further_info=%s, blood_pressure=%s, storage_advice=%s, 
                    medication=%s, patient_id=%s, nhs_number=%s, patient_name=%s, date_of_birth=%s, 
                    patient_address=%s, prescription_text=%s
                WHERE reference_no=%s
                """
                # The data tuple for UPDATE needs the Reference No at the end for the WHERE clause
                update_data = data + (ref_no,)
                cursor.execute(update_query, update_data)
                msg = "Record updated successfully!"
            else:
                # ================= INSERT LOGIC =================
                insert_query = """
                INSERT INTO patient_records (
                    reference_no, name_of_tablet, dose, num_of_tablets, lot_no, issue_date, exp_date,
                    daily_dose, side_effect, further_info, blood_pressure, storage_advice, medication,
                    patient_id, nhs_number, patient_name, date_of_birth, patient_address, prescription_text
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """
                # The data tuple for INSERT needs the Reference No at the beginning
                insert_data = (ref_no,) + data
                cursor.execute(insert_query, insert_data)
                msg = "New record saved successfully!"

            conn.commit()
            messagebox.showinfo("Success", msg)
            self.iView() # Refresh the view
            
        except Exception as e:
            messagebox.showerror("Error", f"An unexpected error occurred: {e}")
        finally:
            if conn and conn.is_connected():
                conn.close()

    def iView(self):
        conn = self.db_connect()
        if conn is None: return

        try:
            cursor = conn.cursor()
            # Select key columns for the Treeview display
            select_query = "SELECT reference_no, name_of_tablet, patient_id, patient_name, date_of_birth, dose FROM patient_records"
            cursor.execute(select_query)
            rows = cursor.fetchall()
            
            # Clear existing data in the Treeview
            self.record_tree.delete(*self.record_tree.get_children())
            
            # Insert new data
            for row in rows:
                self.record_tree.insert("", END, values=row)

        except Exception as e:
            messagebox.showerror("View Error", f"An error occurred while fetching data: {e}")
        finally:
            if conn and conn.is_connected():
                conn.close()
                
    def iLoadDataFromTree(self, event):
        # Function to load selected row data into the input fields
        selected_item = self.record_tree.focus()
        if not selected_item:
            return

        ref_no = self.record_tree.item(selected_item, 'values')[0] # Get Ref No from the selected row
        
        conn = self.db_connect()
        if conn is None: return

        try:
            cursor = conn.cursor()
            # Select ALL columns for detailed view
            select_query = "SELECT * FROM patient_records WHERE reference_no = %s"
            cursor.execute(select_query, (ref_no,))
            data = cursor.fetchone()
            
            if data:
                # Map the database row to the StringVar variables (data[1] to data[18] correspond to columns after record_id)
                variables = [
                    self.RefNo_var, self.NameTablet_var, self.Dose_var, self.NumTablet_var, self.Lot_var,
                    self.IssueDate_var, self.ExpDate_var, self.DailyDose_var, self.SideEffect_var,
                    self.FurtherInfo_var, self.BloodPressure_var, self.Storage_var, self.Medication_var,
                    self.PatientID_var, self.NHSNo_var, self.PatientName_var, self.DOB_var, 
                    self.Address_var
                ]
                
                # Set the StringVar values (data is 0-indexed, skip record_id at data[0])
                for i, var in enumerate(variables):
                    var.set(data[i+1])

                # Set the prescription text separately (data[19] is the prescription_text column)
                self.txtPrescription.delete("1.0", END)
                self.txtPrescription.insert(END, data[19])
                
        except Exception as e:
            messagebox.showerror("Load Error", f"An error occurred while loading data: {e}")
        finally:
            if conn and conn.is_connected():
                conn.close()

    def iDelete(self):
        ref_no = self.RefNo_var.get()
        if not ref_no or "REF" not in ref_no:
            messagebox.showwarning("Warning", "Please load a record to delete first.")
            return
            
        confirm = messagebox.askyesno("Delete", f"Are you sure you want to delete record {ref_no}?")
        if not confirm:
            return

        conn = self.db_connect()
        if conn is None: return

        try:
            cursor = conn.cursor()
            delete_query = "DELETE FROM patient_records WHERE reference_no = %s"
            cursor.execute(delete_query, (ref_no,))
            conn.commit()
            
            if cursor.rowcount > 0:
                messagebox.showinfo("Success", f"Record {ref_no} deleted successfully.")
                self.iClear()
                self.iView()
            else:
                messagebox.showwarning("Not Found", f"Record {ref_no} not found.")

        except Exception as e:
            messagebox.showerror("Delete Error", f"An error occurred while deleting: {e}")
        finally:
            if conn and conn.is_connected():
                conn.close()

    def iClear(self):
        # Clear all StringVars
        for var in [self.RefNo_var, self.Dose_var, self.NumTablet_var, self.Lot_var, self.IssueDate_var, 
                    self.ExpDate_var, self.DailyDose_var, self.SideEffect_var, self.FurtherInfo_var, 
                    self.BloodPressure_var, self.Storage_var, self.Medication_var, self.PatientID_var, 
                    self.NHSNo_var, self.PatientName_var, self.DOB_var, self.Address_var]:
            var.set("")
        
        # Clear the Combobox and Text widget
        self.NameTablet_var.set("Nice")
        self.txtPrescription.delete("1.0", END)
        self.generate_reference_no() # Generate a new reference number
        self.iView() 

    def iExit(self):
        if messagebox.askyesno("Hospital Management System", "Are you sure you want to exit?"):
            self.root.destroy()
            
# =======================================MAIN LOOP================================================

if __name__ == '__main__':
    # Use ThemedTk for better UI
    root = ThemedTk(theme="adapta") # 'adapta' is a clean, modern theme.
    app = Hospital(root)
    app.iView() # Load data on startup
    root.mainloop()

## 🛠️ MySQL Database Setup (hospital_db)

To run this application successfully, you must execute the following SQL script to create the database and table structure.

**Instructions:**
1. Log into your MySQL Command Line Client or MySQL Workbench.
2. Paste and run the entire script below.

-- FILE: hospital_schema.sql

-- 1. Create the database if it doesn't exist
CREATE DATABASE IF NOT EXISTS hospital_db;

-- 2. Use the newly created database
USE hospital_db;

-- 3. Table to store patient records and prescription details
-- All fields from the GUI are mapped here
CREATE TABLE IF NOT EXISTS patient_records (
    record_id INT AUTO_INCREMENT PRIMARY KEY,
    
    -- Medicine Details
    reference_no VARCHAR(50) UNIQUE NOT NULL,
    name_of_tablet VARCHAR(100),
    dose VARCHAR(50),
    num_of_tablets INT,
    lot_no VARCHAR(50),
    issue_date VARCHAR(50), -- Using VARCHAR for dates to match simple entry field
    exp_date VARCHAR(50),   -- Using VARCHAR for dates to match simple entry field
    daily_dose VARCHAR(50),
    side_effect TEXT,
    
    -- Patient Details
    further_info TEXT,
    blood_pressure VARCHAR(50),
    storage_advice VARCHAR(100),
    medication VARCHAR(100),
    patient_id VARCHAR(50),
    nhs_number VARCHAR(50),
    patient_name VARCHAR(100),
    date_of_birth VARCHAR(50), -- Using VARCHAR for dates
    patient_address TEXT,
    
    -- Prescription Text
    prescription_text TEXT
);

-- OPTIONAL: Add a dummy record for testing
INSERT INTO patient_records (
    reference_no, name_of_tablet, dose, num_of_tablets, lot_no, issue_date, exp_date,
    daily_dose, side_effect, further_info, blood_pressure, storage_advice, medication,
    patient_id, nhs_number, patient_name, date_of_birth, patient_address, prescription_text
) VALUES (
    'REF12345', 'Cororna vaccine', '1 dose', 1, 'L456', '10/09/25', '10/09/26', 
    '1 per day', 'none', 'nothing urgent', '120/80', 'dry place', 'Water', 
    'P999', 'NHS007', 'John Smith', '01/01/1980', '123 Fake St.', 'Take medication with food.'
);